# Assignment 4


In this task you have to scrape all the results from "Extranjero". [Here is link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E). 
You have to **follow the setps as we did in Lecture 7** and get information from all tabs in "Continente", "País" and "Estado".   

1. Take the data table TOTAL DE VOTOS. 
2. Store the four columns ORGANIZACIONES POLÍTICAS, TOTAL, %VÁLIDOS and %EMITIDOS. Rename these columns to political_parties, total, share_valid, share_emitied, respectively. 
3. Add the columns continent, country, state, ubigeo, and link. These last two items you can extract from the url. 
4. Append all the tables in just one DataFrame and export an excel file and save it in the folder [\_data\_results](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_7/_data_results). 
5. The name of your Excel file should be the same as the name of your branches. 

**Step 1** Import the librarys


In [3]:
from selenium import webdriver
import re
import time 
from selenium.webdriver.support.ui import Select
from datetime import date
import pandas as pd
import json

**Step 2** Generate the web driver

In [4]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
driver=webdriver.Chrome(executable_path='chromedriver', options=options)
driver.maximize_window()

url="https://resultadoshistorico.onpe.gob.pe/EG2021/"

driver.get(url)

In [5]:
#Solution of the problem : we find a problen with the link, it does't open. So we 
Election=driver.find_element_by_xpath("/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]")
Election.click()

C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/1051048992.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Election=driver.find_element_by_xpath("/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]")


In [7]:
#Select Foreign contries
FOREIGN=Select(driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select"))
For = FOREIGN.options[ 2 ]
For.click()

C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/1510869081.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  FOREIGN=Select(driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select"))


In [8]:
all_tables = {}
Continent_0 = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" )
Continent=Select(Continent_0)
num_Continent=len(Continent.options)
num_Continent


C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/2471109406.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Continent_0 = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" )


6

In [9]:
for Cont_x in range(num_Continent):
    # Get again all Continent since HTML is refreshing
    # all elements
    # Click on one specific Continent
    Continent_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ))
    Continent = Continent_0.options[ Cont_x ]
    #Extract name of Continents text
    Continent_name = Continent.text
   
    # We select a different Continent name
    if Continent_name != "--TODOS--" :
        # click on Continet
        Continent.click()
    
        # Get all elements of Countries
        Country_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ))
        #Number of Countries
        num_Country=len(Country_0.options)
        
        for Coun_x in range(num_Country):
            # Get again all Countries since HTML is refreshing
            # all elements
            Country_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ))   
            Country = Country_0.options[Coun_x]
            #Extract name of Countries text
            Country_name=Country.text
            # We select a different Continent name  
            
            if Country_name != "--TODOS--" :
                # click on Country
                Country.click()
                # Get all elements of State
                State_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ))
                #Number of Countries
                num_State=len(State_0.options)
                #Number of Satates
                num_State=len(State_0.options)
                
                for Sta_x in range(num_State):
                    # Get again all Countries since HTML is refreshing
                    # all elements
                    State_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ))   
                    State = State_0.options[Sta_x]
                    #Extract name of Countries text
                    State_name=State.text
                    # We select a different Continent name
                    
                    if State_name != "--TODOS--":
                        # click on Country
                        State.click()        
                        # Get UBIGEO
                        ubigeo =driver.current_url.split("/")[-1]
                        ## Get table of presidential votes
                        # Get html at this point
                        table_path=driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div")
                        table_html=table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( " ", "_") \
                                              .str.lower().str.replace( "%", "share_") \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        #Change colmns names
                        table_clean=table_clean.rename(columns={"total":"Total","organizaciones_políticas":"Political_parties","share_válidos":"Share_valid", "share_emitidos":"Share_emitied"}, inplace= False)
                                                
                        # New values to columns 
                        table_clean[ 'Continent' ] = Continent_name
                        table_clean[ 'Country' ]   = Country_name
                        table_clean[ 'State' ]   = State_name
                        table_clean[ 'Ubigeo' ]     = ubigeo
                        table_clean[ 'URL' ] = driver.current_url

                        # store tables
                        all_tables[ ubigeo ] = table_clean

C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/2924146598.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Continent_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ))
C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/2924146598.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Country_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ))
C:\Users\Raysa\AppData\Local\Temp/ipykernel_280/2924146598.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Country_0=Select(driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ))
C:\Users\Ray

In [10]:
Final_data = pd.concat( all_tables.values() ).reset_index( drop = True )
Final_data

,Political_parties,Total,Share_valid,Share_emitied,Continent,Country,State,Ubigeo,URL
0,PARTIDO NACIONALISTA PERUANO,0,0.000%,0.000%,AFRICA,SUDAFRICA,PRETORIA,910603,https://resultadoshistorico.onpe.gob.pe/EG2021...
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",0,0.000%,0.000%,AFRICA,SUDAFRICA,PRETORIA,910603,https://resultadoshistorico.onpe.gob.pe/EG2021...
2,PARTIDO MORADO,0,0.000%,0.000%,AFRICA,SUDAFRICA,PRETORIA,910603,https://resultadoshistorico.onpe.gob.pe/EG2021...
3,PERU PATRIA SEGURA,0,0.000%,0.000%,AFRICA,SUDAFRICA,PRETORIA,910603,https://resultadoshistorico.onpe.gob.pe/EG2021...
4,VICTORIA NACIONAL,0,0.000%,0.000%,AFRICA,SUDAFRICA,PRETORIA,910603,https://resultadoshistorico.onpe.gob.pe/EG2021...
...,...,...,...,...,...,...,...,...,...
4091,ALIANZA PARA EL PROGRESO,0,0.000%,0.000%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4092,TOTAL DE VOTOS VÁLIDOS,34,100.000%,97.143%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4093,VOTOS EN BLANCO,0,NaN,0.000%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4094,VOTOS NULOS,1,NaN,2.857%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...


In [12]:
Final_data.to_excel(r'../_data_results/Group_2_ass_4.xlsx', index=False)

In [13]:
driver.close()